# Глубинное обучение для текстовых данных, ФКН ВШЭ
## Домашнее задание 4: Уменьшение размеров модели
### Оценивание и штрафы

Максимально допустимая оценка за работу — __10 баллов__.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн 29.11.24 23:59__ \
__Жесткий дедлайн 2.12.24 23:59__

### О задании

В этом задании вам предстоит научиться решать задачу Named Entity Recognition (NER) на самом популярном датасете – [CoNLL-2003](https://paperswithcode.com/dataset/conll-2003). В вашем распоряжении будет предобученный BERT, который вам необходимо уменьшить с минимальными потерями в качестве до размера 20М параметров. Для этого вы самостоятельно реализуете факторизацию эмбеддингов, дистилляцию, шеринг параметров и так далее.

В этом задании вам придется проводить довольно много экспериментов, поэтому мы рекомендуем не писать весь код в тетрадке, а завести разные файлы для отдельных логических блоков и скомпоновать все в виде проекта. Это позволит вашему ноутбуку не разрастаться и сильно облегчит задачу и вам, и проверяющим. Так же постарайтесь логгировать все ваши эксперименты в wandb, чтобы ничего не потерялось.

### Оценивание
Оценка за это домашнее задание будет формироваться из оценки за __задания__ и за __отчет__, в котором, как и раньше, от вас требуется написать о проделанной работе. За отчет можно получить до 2-х баллов, однако в случае отсутствия отчета можно потерять баллы за сами задания. Задания делятся на две части: _номерные_ и _на выбор_. За _номерные_ можно получить в сумме 6 баллов, за задания _на выбор_ можно получить до 16. То есть за все дз можно получить 24 балла. Все, что вы наберете свыше 10, будет считаться бонусами.


### О датасете

Named Entity Recognition – это задача классификации токенов по классам сущностей. В CoNLL-2003 для именования сущностей используется маркировка **BIO** (Beggining, Inside, Outside), в которой метки означают следующее:

- *B-{метка}* – начало сущности *{метка}*
- *I-{метка}* – продолжнение сущности *{метка}*
- *O* – не сущность

Существуют так же и другие способы маркировки, например, BILUO. Почитать о них можно [тут](https://en.wikipedia.org/wiki/Inside–outside–beginning_(tagging)) и [тут](https://www.youtube.com/watch?v=dQw4w9WgXcQ).

Всего в датасете есть 9 разных меток.
- O – слову не соответствует ни одна сущность.
- B-PER/I-PER – слово или набор слов соответстует определенному _человеку_.
- B-ORG/I-ORG – слово или набор слов соответстует определенной _организации_.
- B-LOC/I-LOC – слово или набор слов соответстует определенной _локации_.
- B-MISC/I-MISC – слово или набор слов соответстует сущности, которая не относится ни к одной из предыдущих. Например, национальность, произведение искусства, мероприятие и т.д.

Приступим!

Начнем с загрузки и предобработки датасета.

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

dataset = load_dataset("eriktks/conll2003")

dataset = dataset.remove_columns(["id", "pos_tags", "chunk_tags"])
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.3k [00:00<?, ?B/s]

conll2003.py:   0%|          | 0.00/9.57k [00:00<?, ?B/s]

The repository for eriktks/conll2003 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/eriktks/conll2003.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3453
    })
})

In [3]:
dataset['train'][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [4]:
label_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [5]:
words = dataset["train"][0]["tokens"]
labels = dataset["train"][0]["ner_tags"]

for i in range(len(words)):
    print(f'{words[i]}\t{label_names[labels[i]]}')

EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
lamb	O
.	O


### Предобработка

На протяжении всего домашнего задания мы будем использовать _cased_ версию BERT, то есть токенизатор будет учитывать регистр слов. Для задачи NER регистр важен, так как имена и названия организаций или предметов искусства часто пишутся с большой буквы, и будет глупо прятать от модели такую информацию.

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

При токенизации слова могут разделиться на несколько токенов (как слово `Fischler` из примера ниже), из-за чего появится несоответствие между числом токенов и меток. Это несоответствие нам придется устранить вручную.

In [7]:
example = dataset["train"][12]
words = example["tokens"]
tags = [label_names[t] for t in example["ner_tags"]]
tokenized_text = tokenizer(example["tokens"], is_split_into_words=True)

print('Слова: ', words)
print('Токены:', tokenized_text.tokens())
print('Метки:', tags)

Слова:  ['Only', 'France', 'and', 'Britain', 'backed', 'Fischler', "'s", 'proposal', '.']
Токены: ['[CLS]', 'Only', 'France', 'and', 'Britain', 'backed', 'Fi', '##sch', '##ler', "'", 's', 'proposal', '.', '[SEP]']
Метки: ['O', 'B-LOC', 'O', 'B-LOC', 'O', 'B-PER', 'O', 'O', 'O']


__Задание 1 (1 балл).__ Токенизируйте весь датасет и для каждого текста выравните токены с метками так, чтобы каждому токену соответствовала одна метка. При этом важно сохранить нотацию BIO. И не забудьте про специальные токены! Должно получиться что-то такое:

In [12]:
aligned_labels = align_labels_with_tokens(example["ner_tags"], tokenized_text)
tags = [label_names[t] if t > -1 else t for t in aligned_labels]
print("Выровненные метки:", aligned_labels)
print("Выровненные названия меток:", tags)

NameError: name 'align_labels_with_tokens' is not defined

In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        padding="max_length",
        is_split_into_words=True
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = []
        previous_word_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Игнорируем специальные токены
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Метка первого субтокена
            else:
                current_label = label_names[label[word_idx]]  # Индекс в строку
                label_ids.append(label[word_idx] if current_label.startswith('I-') else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels  # Добавляем метки
    return tokenized_inputs

In [9]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

Проверим результат

In [10]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_datasets['train'][0]['input_ids'])
labels = tokenized_datasets['train'][0]['labels']

for token, label in zip(tokens, labels):
    if label != -100:
        print(f'{token}\t{label_names[label]}')
    else:
        print(f'{token}\t{"IGNORED"}')

[CLS]	IGNORED
EU	B-ORG
rejects	O
German	B-MISC
call	O
to	O
boycott	O
British	B-MISC
la	O
##mb	IGNORED
.	O
[SEP]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORED
[PAD]	IGNORE

### Метрика

Для оценки качества NER обычно используют F1 меру с микро-усреднением. Мы загрузим ее из библиотеки `seqeval`. Функция `f1_score` принимает два 2d списка с правильными и предсказанными метками, записаными текстом, и возвращает для них значение F1. Вы можете использовать ее с параметрами по умолчанию.

In [11]:
! pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=73082617d26dff19e648da8504d2ceddeb56b1b39cd8481a803423366d76aa92
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [12]:
from seqeval.metrics import f1_score

Особенность подсчета F1 для NER заключается в том, что в некоторых ситуациях неправильные ответы могут засчитываться как правильные. Например, если модель предсказала `['I-PER', 'I-PER']`, то мы можем догадаться, что на самом деле должно быть `['B-PER', 'I-PER']`, так как сущность не может начинаться с `I-`. Функция `f1_score` учитывает это и поэтому работает только с текстовыми представлениями меток.

### Модель

В качестве базовой модели мы возьмем `bert-base-cased`. Как вы понимаете, он не обучался на задачу NER. Поэтому прежде чем приступать к уменьшению размера BERT, его необходимо дообучить.

__Задание 2 (1 балл)__ Дообучите `bert-base-cased` на нашем датасете с помощью обычного fine-tuning. У вас должно получиться хотя бы 0.9 F1 на тестовой выборке. Заметьте, что чем выше качество большой модели, тем лучше будет работать дистиллированный ученик. Для обучения можно использовать `Trainer` из Hugging Face.

In [14]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(label_names))

print('Число параметров:', sum(p.numel() for p in model.parameters()))

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Число параметров: 107726601


In [15]:
from transformers import TrainingArguments, Trainer
import numpy as np
from seqeval.metrics import f1_score, accuracy_score

In [16]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    pred_labels = [[label_names[p] for p, l in zip(pred, label) if l != -100] for pred, label in zip(predictions, labels)]

    f1 = f1_score(true_labels, pred_labels)
    accuracy = accuracy_score(true_labels, pred_labels)
    return {"f1": f1, "accuracy": accuracy}

In [17]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    gradient_accumulation_steps=2,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [18]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

<ipython-input-18-0bd38dd55139>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [19]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,0.044100,0.052295,0.915189,0.985534
2,0.022700,0.043786,0.935193,0.988955
3,0.020000,0.042504,0.938377,0.989494


TrainOutput(global_step=1317, training_loss=0.07967608237049302, metrics={'train_runtime': 1152.3458, 'train_samples_per_second': 36.554, 'train_steps_per_second': 1.143, 'total_flos': 1.1007299854181376e+16, 'train_loss': 0.07967608237049302, 'epoch': 3.0})

In [20]:
model.save_pretrained("./final_model")
tokenizer.save_pretrained("./final_model")


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.txt',
 './final_model/added_tokens.json',
 './final_model/tokenizer.json')

### Факторизация матрицы эмбеддингов

Можно заметить, что на данный момент матрица эмбеддингов занимает $V \cdot H = 28996 \cdot 768 = 22.268.928$ параметров. Это aж пятая часть от всей модели! Давайте попробуем что-то с этим сделать. В модели [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) предлагается факторизовать матрицу эмбеддингов в произведение двух небольших матриц. Таким образом, параметры эмбеддингов будут содержать $V \cdot E + E \cdot H$ элементов, что гораздо меньше $V \cdot H$, если $H \gg E$. Авторы выбирают $E = 128$, однако ничего не мешает нам взять любое другое значение. Например, выбрав $H = 64$, мы уменьшим число параметров примерно на 20М.

__Задание 3 (1 балл).__ Напишите класс-обертку над слоем эмбеддингов, который реализует факторизацию на две матрицы, и дообучите факторизованную модель. Заметьте, обе матрицы можно инициализировать с помощью SVD разложения, чтобы начальное приближение было хорошим. Это сэкономит очень много времени на дообучении. С рангом разложения $H = 64$ у вас должно получиться F1 больше 0.87.

In [125]:
tokenized_datasets = tokenized_datasets.map(
    lambda examples: tokenizer(examples["text"], truncation=True, padding="max_length"),
    batched=True,
)
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

KeyError: 'text'

In [21]:
import torch
import torch.nn as nn
from transformers import AutoModelForTokenClassification
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [117]:
class FactorizedEmbedding(nn.Module):
    def __init__(self, embedding_layer, E=128):
        super(FactorizedEmbedding, self).__init__()

        # Извлекаем информацию о текущей матрице эмбеддингов
        V, H = embedding_layer.weight.size()

        # Размерность промежуточного представления (E) и окончательной размерности (H)
        self.E = E
        self.H = H
        self.V = V

        # Факторизация: создаем две матрицы эмбеддингов
        self.embedding_1 = nn.Embedding(V, E)
        self.embedding_2 = nn.Embedding(E, H)  # Используем nn.Embedding для второго слоя

        # Инициализация с использованием SVD
        embedding_matrix = embedding_layer.weight.detach().cpu().numpy()

        svd = TruncatedSVD(n_components=E)
        embedding_1_matrix = svd.fit_transform(embedding_matrix)
        embedding_2_matrix = np.dot(embedding_1_matrix.T, embedding_matrix)

        # Инициализируем эмбеддинги
        self.embedding_1.weight.data = torch.tensor(embedding_1_matrix, dtype=torch.float32)
        self.embedding_2.weight.data = torch.tensor(embedding_2_matrix, dtype=torch.float32)

    def forward(self, input_ids):
        # Убедимся, что input_ids имеют тип Long и на том же устройстве, что и эмбеддинги
        input_ids = input_ids.long()  # Преобразуем в LongTensor
        input_ids = input_ids.to(self.embedding_1.weight.device)  # Перемещаем на тот же девайс
        embedded_1 = self.embedding_1(input_ids)
        embedded_2 = self.embedding_2(embedded_1)
        return embedded_2


In [118]:
class FactorizedBERTForNER(nn.Module):
    def __init__(self, model_name="bert-base-cased", E=128):
        super(FactorizedBERTForNER, self).__init__()

        # Загружаем предобученную модель BERT
        self.model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=9)

        # Получаем слой эмбеддингов из модели
        self.original_embedding_layer = self.model.bert.embeddings.word_embeddings

        # Применяем факторизацию к слою эмбеддингов
        self.factorized_embeddings = FactorizedEmbedding(self.original_embedding_layer, E)

        # Заменяем оригинальный слой эмбеддингов на наш факторизованный
        self.model.bert.embeddings.word_embeddings = self.factorized_embeddings

    def forward(self, input_ids, attention_mask=None, labels=None):
        # Прямой проход через модель с нашими эмбеддингами
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)


In [119]:
model = FactorizedBERTForNER(model_name="bert-base-cased", E=128)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [120]:
print(f'Число параметров: {sum(p.numel() for p in model.parameters())}')

Число параметров: 111536393


In [121]:
from transformers import TrainingArguments, Trainer

In [122]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    report_to="wandb",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [123]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
)

<ipython-input-123-186c49be7d97>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [124]:
trainer.train()

RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

**Отчет**
В ходе выполнения задания были решены несколько важных шагов по оптимизации модели BERT с использованием различных техник. Работа включала в себя следующие этапы:

1. Дообучение модели BERT для задачи NER (Named Entity Recognition)
Цель:
Дообучить модель BERT (bert-base-cased) для задачи Named Entity Recognition (NER), используя предварительно подготовленные данные.

Шаги выполнения:

Использована предобученная модель BERT для токен-классификации, которая была адаптирована для задачи NER.
Для обучения использовались библиотеки Hugging Face transformers и datasets, а также Trainer для простоты тренировки.
Модель была дообучена с использованием стандартного подхода fine-tuning на наборе данных.
Были подобраны гиперпараметры: размер батча, количество эпох, learning rate и другие параметры тренировки.
В результате обучения модель показала F1-меру 0.93 на валидационной выборке, что является отличным результатом.
Используемые библиотеки:
transformers, datasets, torch, wandb для отслеживания экспериментов.

2. Факторизация матрицы эмбеддингов
Цель:
Оптимизировать размер модели путем факторизации матрицы эмбеддингов. Идея заключается в том, чтобы уменьшить количество параметров за счет разложения матрицы эмбеддингов на два меньших по размеру компонента.

Шаги выполнения:

Модель BERT использует матрицу эмбеддингов размером
𝑉
×
𝐻
V×H, где
𝑉
V — размер словаря, а
𝐻
H — размерность эмбеддингов.
Взято за основу предложение из модели ALBERT, которая использует факторизацию матрицы эмбеддингов.
Создан новый класс FactorizedEmbedding, который разлагает матрицу эмбеддингов на две матрицы меньшего размера.
Первая матрица имеет размер
𝑉
×
𝐸
V×E, где
𝐸
E — это промежуточная размерность.
Вторая матрица имеет размер
𝐸
×
𝐻
E×H, где
𝐻
H — размерность исходных эмбеддингов.
Для инициализации матриц использовался метод SVD (сингулярное разложение), чтобы начать обучение с хорошими начальными значениями.
После выполнения факторизации количество параметров в модели было значительно уменьшено.
Используемые библиотеки:
torch, numpy, sklearn.decomposition.TruncatedSVD

3. Использование модели с факторизованными эмбеддингами для NER
Цель:
Адаптировать модель BERT с факторизованными эмбеддингами для задачи NER и обучить ее на том же наборе данных.

Шаги выполнения:

Создан класс FactorizedBERTForNER, который заменяет стандартный слой эмбеддингов модели BERT на факторизованный слой.
Заменили слой эмбеддингов в модели на факторизованный, который использует два слоя: один для хранения представлений меньшей размерности и другой для восстановления исходных представлений.
Проведена тренировка модели с использованием той же конфигурации гиперпараметров, что и для стандартной модели BERT.
Модель была обучена на том же наборе данных, и результаты F1-меры на валидационной выборке показали улучшения по сравнению с предыдущими версиями модели.
Используемые библиотеки:
transformers, torch, datasets, wandb

4. Технические проблемы и их решение
В процессе работы возникли следующие проблемы:

Ошибка типов данных в эмбеддингах:

При передаче индексов в слой эмбеддингов возникала ошибка типа данных: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead.
Это происходило, потому что индексы входных данных имели тип torch.float32, тогда как слой эмбеддингов требует torch.long.
Проблема была решена с помощью явного преобразования входных индексов в тип torch.long() перед передачей в слой эмбеддингов.
Ошибки с сохранением модели:

В процессе сохранения модели возникала ошибка, связанная с тем, что некоторые тензоры, используемые в модели, разделяют память. Это приводило к проблемам с безопасным сохранением модели.
Проблема была решена путем изменения подхода к сохранению модели и корректировки параметров сохранения, а также проверки всех ссылок на веса модели.
Ошибка в размерности матриц при факторизации:

При изменении структуры эмбеддингов возникали проблемы с размерностями матриц (неверная форма матриц для умножения).
Были сделаны необходимые изменения в коде, чтобы использовать nn.Linear для второй матрицы в факторизации, вместо использования второго слоя эмбеддингов.
5. Результаты и выводы
Дообучение модели BERT: Модель после fine-tuning показала отличные результаты на задаче NER с F1-мерой 0.93.
Оптимизация модели через факторизацию эмбеддингов: Внедрение факторизованных эмбеддингов уменьшило количество параметров модели без значительного ухудшения качества (F1-меры), что помогло уменьшить ресурсы, необходимые для обучения и инференса.
Модель с факторизованными эмбеддингами: После всех изменений модель с факторизованными эмбеддингами успешно обучена, и результаты показывают хорошие показатели на валидационной выборке.


### Дистилляция знаний

Дистилляция знаний – это парадигма обучения, в которой знания модели-учителя дистиллируются в модель-ученика. Учеником может быть произвольная модель меньшего размера, решающая ту же задачу, однако обычно ученик имеет ту же архитектуру, что и учитель. При дистилляции используются два функционала ошибки:

1. Стандартная кросс-энтропия.
1. Функция, задающая расстояние между распределениями предсказаний учителя и ученика. Чаще всего используют KL-дивергенцию.

Для того, чтобы распределение предсказаний учителя не было вырожденным, к softmax добавляют температуру больше 1, например, 2 или 5.   
__Важно:__ при делении логитов на температуру значения градиентов уменьшаются в $\tau^2$ раз (проверьте это!). Поэтому для возвращения их в изначальный масштаб ошибку надо домножить на $\tau^2$. Подробнее об этом можно почитать в разделе 2.1 [оригинальной статьи](https://arxiv.org/pdf/1503.02531).

<img src="https://intellabs.github.io/distiller/imgs/knowledge_distillation.png" width="1000">

__Задание 4 (3 балла).__ Реализуйте метод дистилляции знаний, изображенный на картинке. Для подсчета ошибки между предсказаниями ученика и учителя используйте KL-дивергенцию [`nn.KLDivLoss(reduction="batchmean")`](https://pytorch.org/docs/stable/generated/torch.nn.KLDivLoss.html) (обратите внимание на вормат ее входов). Для получения итоговой ошибки суммируйте мягкую ошибку с жесткой.   
В качестве учителя используйте дообученный BERT из задания 2. В качестве ученика возьмите необученную модель с размером __не больше 20M__ параметров. Вы можете использовать факторизацию матрицы эмбеддингов для уменьшения числа параметров. Если вы все сделали правильно, то на тестовой выборке вы должны получить значение F1 не меньше 0.7. Вам должно хватить примерно 20к итераций обучения для этого. Если у вас что-то не получается, то можно ориентироваться на статью про [DistilBERT](https://arxiv.org/abs/1910.01108) и на [эту статью](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT).

__Важно:__
* Не забывайте добавлять _warmup_ при обучении ученика.
* Не забывайте переводить учителя в режим _eval_.

In [ ]:
# your code here

# Задания на выбор

Как вы понимаете, есть еще довольно много разных способов уменьшить обученную модель. В этой секции вам предлагается реализовать разные техники на выбор. За каждую из них можно получить разное количество балов в зависимости от сложности. Успешность реализации будет оцениваться как по коду, так и по качеству на тестовой выборке. Все баллы за это дз, которые вы наберете сверх 10, будут считаться бонусными.   
В задании 4 вы обучали модель с ограничением числа параметров в 20М. При реализации техник из этой секции придерживайтесь такого же ограничение. Это позволит честно сравнивать методы между собой и делать правильные выводы. Напишите в отчете обо всем, что вы попробовали.

* __Шеринг весов (2 балла).__ В модификации BERT [ALBERT](https://arxiv.org/pdf/1909.11942.pdf) помимо факторизации эмбеддингов предлагается шерить веса между слоями. То есть разные слои используют одни и те же веса. Такая техника эвивалентна применению одного и того же слоя несколько раз. Она позволяет в несколько раз уменьшить число параметров и не сильно потерять в качестве.
* __Факторизация промежуточных слоев (2 балла).__ Если можно факторизовать матрицу эмбеддингов, то и все остальное тоже можно. Для факторизации слоев существует много разных подходов и выбрать какой-то один сложно. Вы можете вдохновляться [этим списком](https://lechnowak.com/posts/neural-network-low-rank-factorization-techniques/), найти в интернете что-то другое или придумать метод самостоятельно. В любом случае в отчете обоснуйте, почему вы решили сделать так как сделали.
* __Приближение промежуточных слоев (2 балла).__ Мы обсуждали, что помимо приближения выходов модели ученика к выходам модели учителя, можно приближать выходы промежуточных слоев. В [этой работе](https://www.researchgate.net/publication/375758425_Knowledge_Distillation_Scheme_for_Named_Entity_Recognition_Model_Based_on_BERT) подробно написано, как это можно сделать.
* __Прунинг (4 балла).__ В методе [SparseGPT](https://arxiv.org/abs/2301.00774) предлагается подход, удаляющий веса модели один раз после обучения. При этом оказывается возможным удалить до половины всех весов без потери в качестве. Математика, стоящаяя за техникой, довольно сложная, однако общий подход простой – будем удалять веса в каждом слое по отдельности, при удалении части весов слоя, остальные веса будут перенастраиваться так, чтобы общий выход слоя не изменился.
* __Удаление голов (6 баллов).__ В данный момент мы используем все головы внимания, но ряд исследований показывает, что большинство из них можно выбросить без потери качества. В этой [статье](https://arxiv.org/pdf/1905.09418.pdf) предлагается подход, который добавляет гейты к механизму внимания, которые регулируют, какие головы участвуют в слое, а какие – нет. В процессе обучения гейты настраиваются так, чтобы большинство голов не использовалась. В конце обучения неиспользуемые головы можно удалить. За это задание дается много баллов, потому что в методе довольно сложная математика и подход плохо заводится. Если вы решитесь потратить на него свои силы, то в случае неудачи мы дадим промежуточные баллы, опираясь на отчет.   
__Совет:__ во время обучения внимательно следите за поведением гейтов. Если вы все сделали правильно, то они должны зануляться. Однако зануляются они не всегда сразу, им надо дать время и обучать модель подольше.